# Vectorizing

In [1]:
a = [1,2,3]
b = [4,5,6]

a .* b

3-element Vector{Int64}:
  4
 10
 18

Dimensions should match:

In [2]:
[1, 2, 3] .* [6, 5]

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 3 and 2

However, one element collections will be broadcast in obvious way

In [3]:
[1, 2, 3] .* [5]

3-element Vector{Int64}:
  5
 10
 15

In [5]:
[1,2,3] * [1 2 3]

3×3 Matrix{Int64}:
 1  2  3
 2  4  6
 3  6  9

But:

In [7]:
[1 2 3] * [1,2,3]

1-element Vector{Int64}:
 14

In [8]:
[1, 2, 3] .=> [0, 4, 1]

3-element Vector{Pair{Int64, Int64}}:
 1 => 0
 2 => 4
 3 => 1

In [10]:
?2 => 4

```
Pair(x, y)
x => y
```

Construct a `Pair` object with type `Pair{typeof(x), typeof(y)}`. The elements are stored in the fields `first` and `second`. They can also be accessed via iteration (but a `Pair` is treated as a single "scalar" for broadcasting operations).

See also [`Dict`](@ref).

# Examples

```jldoctest
julia> p = "foo" => 7
"foo" => 7

julia> typeof(p)
Pair{String, Int64}

julia> p.first
"foo"

julia> for x in p
           println(x)
       end
foo
7
```


In [11]:
[1, 2, 3] .>= [0, 4, 1]

3-element BitVector:
 1
 0
 1

In [12]:
1 in [1,2,3]

true

In [13]:
12 in [1,2,3]

false

In [14]:
in(1, [1,2,3])

true

> you can add a dot (.) before any operator to broadcast it. What about functions that are not operators? Here you also use a dot (.), but this time, you suffix it after the function name.

In [15]:
abs.([1, -2, 3, -6, -14])

5-element Vector{Int64}:
  1
  2
  3
  6
 14

In [17]:
?Ref(1)

```
Ref{T}
```

An object that safely references data of type `T`. This type is guaranteed to point to valid, Julia-allocated memory of the correct type. The underlying data is protected from freeing by the garbage collector as long as the `Ref` itself is referenced.

In Julia, `Ref` objects are dereferenced (loaded or stored) with `[]`.

Creation of a `Ref` to a value `x` of type `T` is usually written `Ref(x)`. Additionally, for creating interior pointers to containers (such as Array or Ptr), it can be written `Ref(a, i)` for creating a reference to the `i`-th element of `a`.

`Ref{T}()` creates a reference to a value of type `T` without initialization. For a bitstype `T`, the value will be whatever currently resides in the memory allocated. For a non-bitstype `T`, the reference will be undefined and attempting to dereference it will result in an error, "UndefRefError: access to undefined reference".

To check if a `Ref` is an undefined reference, use [`isassigned(ref::RefValue)`](@ref). For example, `isassigned(Ref{T}())` is `false` if `T` is not a bitstype. If `T` is a bitstype, `isassigned(Ref{T}())` will always be true.

When passed as a `ccall` argument (either as a `Ptr` or `Ref` type), a `Ref` object will be converted to a native pointer to the data it references. For most `T`, or when converted to a `Ptr{Cvoid}`, this is a pointer to the object data. When `T` is an `isbits` type, this value may be safely mutated, otherwise mutation is strictly undefined behavior.

As a special case, setting `T = Any` will instead cause the creation of a pointer to the reference itself when converted to a `Ptr{Any}` (a `jl_value_t const* const*` if T is immutable, else a `jl_value_t *const *`). When converted to a `Ptr{Cvoid}`, it will still return a pointer to the data region as for any other `T`.

A `C_NULL` instance of `Ptr` can be passed to a `ccall` `Ref` argument to initialize it.

# Use in broadcasting

`Ref` is sometimes used in broadcasting in order to treat the referenced values as a scalar.

# Examples

```jldoctest
julia> Ref(5)
Base.RefValue{Int64}(5)

julia> isa.(Ref([1,2,3]), [Array, Dict, Int]) # Treat reference values as scalar during broadcasting
3-element BitVector:
 1
 0
 0

julia> Ref{Function}()  # Undefined reference to a non-bitstype, Function
Base.RefValue{Function}(#undef)

julia> try
           Ref{Function}()[] # Dereferencing an undefined reference will result in an error
       catch e
           println(e)
       end
UndefRefError()

julia> Ref{Int64}()[]; # A reference to a bitstype refers to an undetermined value if not given

julia> isassigned(Ref{Int64}()) # A reference to a bitstype is always assigned
true

julia> Ref{Int64}(0)[] == 0 # Explicitly give a value for a bitstype reference
true
```


In [18]:
in.([1, 3, 5, 7, 9, 11], Ref([1, 2, 3, 4, 5]))

6-element BitVector:
 1
 1
 1
 0
 0
 0

In [19]:
aq = [10.0   8.04  10.0  9.14  10.0   7.46   8.0   6.58
                      8.0   6.95   8.0  8.14   8.0   6.77   8.0   5.76
                     13.0   7.58  13.0  8.74  13.0  12.74   8.0   7.71
                      9.0   8.81   9.0  8.77   9.0   7.11   8.0   8.84
                     11.0   8.33  11.0  9.26  11.0   7.81   8.0   8.47
                     14.0   9.96  14.0  8.1   14.0   8.84   8.0   7.04
                      6.0   7.24   6.0  6.13   6.0   6.08   8.0   5.25
                      4.0   4.26   4.0  3.1    4.0   5.39  19.0  12.50
                     12.0  10.84  12.0  9.13  12.0   8.15   8.0   5.56
                      7.0   4.82   7.0  7.26   7.0   6.42   8.0   7.91
                      5.0   5.68   5.0  4.74   5.0   5.73   8.0   6.89]

11×8 Matrix{Float64}:
 10.0   8.04  10.0  9.14  10.0   7.46   8.0   6.58
  8.0   6.95   8.0  8.14   8.0   6.77   8.0   5.76
 13.0   7.58  13.0  8.74  13.0  12.74   8.0   7.71
  9.0   8.81   9.0  8.77   9.0   7.11   8.0   8.84
 11.0   8.33  11.0  9.26  11.0   7.81   8.0   8.47
 14.0   9.96  14.0  8.1   14.0   8.84   8.0   7.04
  6.0   7.24   6.0  6.13   6.0   6.08   8.0   5.25
  4.0   4.26   4.0  3.1    4.0   5.39  19.0  12.5
 12.0  10.84  12.0  9.13  12.0   8.15   8.0   5.56
  7.0   4.82   7.0  7.26   7.0   6.42   8.0   7.91
  5.0   5.68   5.0  4.74   5.0   5.73   8.0   6.89

In [20]:
using Statistics

In [21]:
mean.(eachcol(aq))

8-element Vector{Float64}:
 9.0
 7.500909090909093
 9.0
 7.500909090909091
 9.0
 7.500000000000001
 9.0
 7.50090909090909

In [24]:
# function R2(x, y)
# X = [ones(11) x]
#     model = X \ y
#     prediction = X * model
#     error = y - prediction
#     SS_res = sum(v -> v ^ 2, error)
#     mean_y = mean(y)
#     SS_tot = sum(v -> (v - mean_y) ^ 2, y)
#     return 1 - SS_res / SS_tot
# end

# becomes:

function R2(x, y)
X = [ones(11) x]
    model = X \ y
    prediction = X * model
    SS_res = sum((y .- prediction) .^ 2)
    SS_tot = sum((y .- mean(y)) .^ 2)
    return 1 - SS_res / SS_tot
end

R2 (generic function with 1 method)